### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []
for page_num in range(1, 400):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/bao-hiem-xa-hoi-bao-hiem-y-te-bao-hiem-that-nghiep/t1/1/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_bh = pd.DataFrame(articles_data)
df_bh.head(5)

,Name,Time,Question,Situation,Answer
0,Phạm Thị Thu Trang,12:09 10/04/2025,Xác định mức hưởng lương hưu từ ngày 1/7/2025,"Mẹ tôi 57 tuổi, đóng BHXH được 5 năm 7 tháng, ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
1,Bùi Thanh Tuấn,11:37 05/04/2025,"Mức đóng BHXH, BHYT của cán bộ không chuyên tr...","Tôi là Phó Chỉ huy trưởng quân sự xã, phụ cấp ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
2,Huỳnh Thị Thơm,10:06 04/04/2025,Làm việc bao nhiêu ngày trong tháng thì phải đ...,- Ngày nghỉ hằng tuần của người lao động là Ch...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
3,Nguyễn Thị Thanh Trung,10:05 02/04/2025,Viên chức đi học nước ngoài có được hưởng chế ...,"Tôi xin hỏi, viên chức được cử đi học nước ngo...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
4,Diệu Minh Hiền,09:17 01/04/2025,"Công ty ngừng hoạt động, làm thế nào để hưởng ...","Tôi đang mang thai tháng thứ 7, công ty tôi ng...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../Source_Fine-Tuning/data/bhxh_bhyt_bhtn_data.csv'
df_bh.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../Source_Fine-Tuning/data/bhxh_bhyt_bhtn_data.csv


In [6]:
df_bh=pd.read_csv("../Source_Fine-Tuning/data/bhxh_bhyt_bhtn_data.csv",encoding='utf-8')

In [7]:
df_bh.shape

(1892, 6)